In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
from tqdm.notebook import tqdm
import json
import functools

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
from utils.fweg import FWEG

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]

print(RANDOM_SEED)

PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')
RESULTS_DIR = os.path.join(BASE_DIR, "results")


55


In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "train.csv"))
hyper_train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_train.csv"))
val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "val.csv"))
hyper_val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_val.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DIR, "test.csv"))

train_full_df = pd.concat([train_df, hyper_train_df])

In [6]:
x_train_full = train_full_df.drop('label', axis=1).values
y_train_full = train_full_df['label'].values

x_hyper_train = hyper_train_df.drop('label', axis=1).values
y_hyper_train = hyper_train_df['label'].values

x_val = val_df.drop('label', axis=1).values
y_val = val_df['label'].values

x_hyper_val = hyper_val_df.drop('label', axis=1).values
y_hyper_val = hyper_val_df['label'].values

x_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train_full.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])

In [8]:
def get_acc(model, x, y):
    preds = utils.utils.compute_preds(
        model,
        x,
        batch_size=BATCH_SIZE,
    )
    return float((np.argmax(preds, axis=1) == y).mean())

def get_loss(model, x, y):
    loss = tf.keras.losses.CategoricalCrossentropy()
    y = tf.keras.utils.to_categorical(y)
    preds = model.predict(x, batch_size=BATCH_SIZE)
    return float(loss(y, preds).numpy())

In [9]:
print("Evaluating base model")
METRIC = 'Accuracy'
batches = []
hyper_train_acc = []
val_acc = []
hyper_val_acc = []
test_acc = []
hyper_train_loss = []
val_loss = []
hyper_val_loss = []
test_loss = []

for batch in range(0, 1300, 100):
    batches.append(batch)
    model.load_weights( os.path.join(MODELS_DIR, f"adult_periodic_base_batch={batch}.h5") )
    
    hta = get_acc(model, x_hyper_train, y_hyper_train)
    va = get_acc(model, x_val, y_val)
    hva = get_acc(model, x_hyper_val, y_hyper_val)
    ta = get_acc(model, x_test, y_test)
    
    htl = get_loss(model, x_hyper_train, y_hyper_train)
    vl = get_loss(model, x_val, y_val)
    hvl = get_loss(model, x_hyper_val, y_hyper_val)
    tl = get_loss(model, x_test, y_test)
    
    hyper_train_acc.append(hta)
    val_acc.append(va)
    hyper_val_acc.append(hva)
    test_acc.append(ta)
    
    hyper_train_loss.append(htl)
    val_loss.append(vl)
    hyper_val_loss.append(hvl)
    test_loss.append(tl)
    

Evaluating base model


In [22]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)

Results file exists, appending to it...


In [11]:
# save results
for i in range(len(batches)):
    extra = {
        'batch': batches[i],
        'train_acc': hyper_train_acc[i],
        'train_loss': hyper_train_loss[i],
        'val_loss': val_loss[i],
        'test_loss': test_loss[i]
    }
    saver.save(
        RANDOM_SEED,
        METRIC,
        "base",
        val_acc[i],
        hyper_val_acc[i],
        test_acc[i],
        json.dumps(extra)
    )

In [12]:
def get_basis_fns(
    train_df,
    val_df,
    hyper_val_df,
    test_df,
    groups,
    add_all
    ):
    """
    Get the basis functions for adult.
    """
    all_groups = [
        [],
        ['private_workforce', 'non_private_workforce'],
        ['private_workforce', 'non_private_workforce', 'income']
    ]
    assert groups in all_groups, f"got unexpected groups {groups}"
    if len(groups) == 0:
        assert add_all is True
        
    np.random.seed(RANDOM_SEED)
    
    if len(groups) == 0:
        basis_train = pd.DataFrame(np.ones(len(train_df)), columns=["All"])
        basis_val = pd.DataFrame(np.ones(len(val_df)), columns=["All"])
        basis_hyper_val = pd.DataFrame(np.ones(len(hyper_val_df)), columns=["All"])
        basis_test = pd.DataFrame(np.ones(len(test_df)), columns=["All"])
    else:
        basis_train = train_df[groups].copy()
        basis_val = val_df[groups].copy()
        basis_hyper_val = hyper_val_df[groups].copy()
        basis_test = test_df[groups].copy()
    
        if add_all:
            basis_train['All'] = 1.0
            basis_val['All'] = 1.0
            basis_hyper_val['All'] = 1.0
            basis_test['All'] = 1.0
        
    return basis_train, basis_val, basis_hyper_val, basis_test


In [25]:
CLASSES = 2
NUM_ITERS = 100
METRIC = "Accuracy"

groups_list = [
    ['private_workforce', 'non_private_workforce', 'income']
]
groups_descr_list = [
    ["workforce, non-private workforce, income",]
]
add_all_list = [False, True]
epsilon_list = [0.0001, 0.001]
use_linear_val_metric_list = [False]

# this fills in most of the arguments for our basis function creator
# it is missing the `groups` arg and `add_all`. FWEG_Hyperparameter_Search
# is given basis_fn_generator and will fill these in as it iterates over
# the hyperparameters.
basis_fn_generator = functools.partial(
    get_basis_fns,
    train_df = train_full_df,
    val_df = val_df,
    hyper_val_df = hyper_val_df,
    test_df = test_df,
)

In [26]:
mock_saver = utils.record.Mock_Recorder()
fweg_hp_s = utils.fweg.FWEG_Hyperparameter_Search(
    mock_saver,
    CLASSES,
    NUM_ITERS,
    METRIC,
    basis_fn_generator,
    groups_list,
    groups_descr_list,
    add_all_list,
    epsilon_list,
    use_linear_val_metric_list,
    RANDOM_SEED,
    use_convergence=True
)

In [27]:
def run_fweg(model):
    preds_train_full = model.predict(x_train_full)
    preds_val = model.predict(x_val)
    preds_hyper_val = model.predict(x_hyper_val)
    preds_test = model.predict(x_test)
    
     # search for best params
    (best_groups, best_add_all, best_epsilon, best_use_linear_val_metric) = fweg_hp_s.search(
        preds_train_full,
        y_train_full,
        preds_val,
        y_val,
        preds_hyper_val,
        y_hyper_val,
        preds_test,
        y_test,
    )
    
    print(best_groups)
    
    # evaluate best params
    basis_train_full, basis_val, basis_hyper_val, basis_test = get_basis_fns(
        train_full_df,
        val_df,
        hyper_val_df,
        test_df,
        best_groups,
        best_add_all
    )
    fweg = utils.fweg.FWEG(
        METRIC,
        NUM_ITERS,
        best_epsilon,
        CLASSES,
        best_use_linear_val_metric,
        RANDOM_SEED,
    )
    
    val_train_list, grad_norm_list, cond_list = fweg.fit(
        preds_train_full,
        y_train_full,
        basis_train_full,
        preds_val,
        y_val,
        basis_val,
    )
    # apply to hyper val set
    preds_hyper_val_list, mval_hyper_val_list = fweg.predict(
        preds_hyper_val,
        y_hyper_val,
        basis_hyper_val,
        deterministic=False,
    )
    start = len(mval_hyper_val_list)//2
    best_idx = start + np.argmax(mval_hyper_val_list[start:])
    # apply to test set
    preds_test_list, mval_test_list = fweg.predict(
        preds_test,
        y_test,
        basis_test,
        deterministic=False,
    )
    
    return {
        'val_score': val_train_list[best_idx],
        'hyper_val_score': mval_hyper_val_list[best_idx],
        'test_score': mval_test_list[best_idx],
    }
    

In [33]:
print("Evaluating fweg models")
fweg_results = dict()

for batch in range(0, 1300, 100):
    model.load_weights( os.path.join(MODELS_DIR, f"adult_periodic_base_batch={batch}.h5") )
    fweg_model_results = run_fweg(model)
    fweg_results[batch] = fweg_model_results

In [31]:
for batch in fweg_results:
    saver.save(
        RANDOM_SEED,
        METRIC,
        "fweg",
        fweg_results[batch]['val_score'],
        fweg_results[batch]['hyper_val_score'],
        fweg_results[batch]['test_score'],
        json.dumps({'batch': batch}),
    )

In [32]:
saver.close()